In [23]:
#  ! pip3 install trulens_eval openai langchain langchain-openai langchain_community faiss-cpu bs4 tiktoken
# !pip3 install langchainhub
# !pip3 install "ipython>=8.12.0" "ipywidgets>=8.0.6"

In [24]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Loading

In [50]:
# !pip3 install -qU langchain-community beautifulsoup4


In [52]:
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()
# print(docs[0].page_content)   
import re
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup


def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()


loader = RecursiveUrlLoader("https://en.wikipedia.org/wiki/Illuminati", extractor=bs4_extractor)
docs = loader.load()
print(docs[0].page_content)

Illuminati - Wikipedia

Jump to content

Main menu

Main menu
move to sidebar
hide

		Navigation
	

Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate

		Contribute
	

HelpLearn to editCommunity portalRecent changesUpload file

Search

Search

Appearance

Create account

Log in

Personal tools

 Create account Log in

		Pages for logged out editors learn more

ContributionsTalk

Contents
move to sidebar
hide

(Top)

1
History

Toggle History subsection

1.1
Origins

1.2
Transition

1.3
Reform

1.3.1
Adolph Knigge

1.3.2
Internal problems

1.3.3
New system

1.4
Attempts at expansion

1.4.1
Convent of Wilhelmsbad

1.4.2
Aftermath of Wilhelmsbad

1.5
Zenith

1.6
Conflict with Rosicrucians

1.7
Internal dissent

1.8
Decline

2
Barruel and Robison

3
Modern Illuminati

4
Legacy

5
References

Toggle References subsection

5.1
Works cited

6
Further reading

7
External links

Toggle the table of contents

Illuminati

74 languages

AfrikaansAlemannischالعربيةAsturian

# Create Vector Store

In [53]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print(documents)
vectorstore = FAISS.from_documents(documents, embeddings)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Illuminati', 'content_type': 'text/html; charset=UTF-8', 'title': 'Illuminati - Wikipedia', 'language': 'en'}, page_content='Illuminati - Wikipedia\n\nJump to content\n\nMain menu\n\nMain menu\nmove to sidebar\nhide\n\n\t\tNavigation\n\t\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\t\tContribute\n\t\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\nSearch\n\nSearch\n\nAppearance\n\nCreate account\n\nLog in\n\nPersonal tools\n\n Create account Log in\n\n\t\tPages for logged out editors learn more\n\nContributionsTalk\n\nContents\nmove to sidebar\nhide\n\n(Top)\n\n1\nHistory\n\nToggle History subsection\n\n1.1\nOrigins\n\n1.2\nTransition\n\n1.3\nReform\n\n1.3.1\nAdolph Knigge\n\n1.3.2\nInternal problems\n\n1.3.3\nNew system\n\n1.4\nAttempts at expansion\n\n1.4.1\nConvent of Wilhelmsbad\n\n1.4.2\nAftermath of Wilhelmsbad\n\n1.5\nZenith\n\n1.6\nConflict with Rosicrucians\n\n1.

# Create RAG

In [54]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Send your request

In [55]:
rag_chain.invoke("Which notable literary figures and nobility were attracted to the Illuminati?")

"Johann Wolfgang von Goethe, Johann Gottfried Herder, and the reigning Duke of Gotha and of Weimar were notable literary figures and nobility attracted to the Illuminati. The Illuminati also had a role in Gothic literature, with a scandalous vogue for German tales of the Illuminati. Mary Shelley's familiarity with the early anti-Illuminati text influenced her work in Frankenstein."

# Initialize Feedback Function(s)

In [56]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on_input_output()
)
# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content .


# Instrument chain for logging with Trulens

In [57]:
tru_recorder = TruChain(rag_chain,
    app_id='Chainillu_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [58]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("Which notable literary figures and nobility were attracted to the Illuminati?")

display(llm_response)

"Notable literary figures and nobility attracted to the Illuminati included Johann Wolfgang von Goethe, Johann Gottfried Herder, and the reigning Duke of Gotha and of Weimar. The Illuminati also had a role in Gothic literature, with writers like Mary Shelley incorporating Illuminati-influenced ideas in works like Frankenstein. The Illuminati have been referenced in various modern works of literature, such as Dan Brown's Angels & Demons and Umberto Eco's Foucault's Pendulum."

In [59]:
tru.get_leaderboard()


,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Chain1_ChatApplication,0.475,0.966667,1.0,1.0,0.004968
Chainillu_ChatApplication,NaN,1.000000,NaN,1.0,0.004295


In [60]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, "Context Relevance")

,question,context,ret
0,Which notable literary figures and nobility we...,Books and comics[edit]\nGothic literature had ...,0.8
1,Which notable literary figures and nobility we...,Zenith\nAlthough their hopes of mass recruitme...,0.9
2,Which notable literary figures and nobility we...,"Barruel and Robison\nBetween 1797 and 1798, Au...",0.8
3,Which notable literary figures and nobility we...,18th-century Bavarian secret society\nThis art...,0.9
